In [1]:
import json
import requests
import re
from IPython import display
from datetime import datetime
import pandas as pd
import time

In [2]:
FIREBASE_PRIVATE_KEY = r'C:\Users\johnp\Documents\ESportsAnalytics\esportsanalytics-3b04cf0c1d47.json'
BASE_URL = 'https://api.opendota.com/api/'
TEAMS = 'teams'
HEROES = 'heroes'
PROPLAYERS = 'proPlayers'
PROMATCHES = 'proMatches'
MATCHES = 'matches'
PLAYERS = 'players'
HEROES_STATS = 'heroStats'

In [3]:
# read team OG matches file
with open(r'C:\Users\johnp\Documents\ESportsAnalytics\team_og_matches.json', 'r', encoding="utf-8") as input_file:

    data = input_file.read()
    team_og_matches_json = json.loads(data)

In [4]:
def match_outcome(radiant_win, radiant):
    if ((radiant_win == False and radiant == False) or (radiant_win == True and radiant == True)):
        return True
    
    elif ((radiant_win == True and radiant == False) or (radiant_win == False and radiant == True)):
        return False

In [43]:
# Convert the team matches json to a dataframe
def team_matches_df(team_matches_json, predict_team_name, predict_team_id):
    
    all_matches_df = pd.DataFrame(data = [(datetime.fromtimestamp(item['start_time']).strftime('%Y-%m-%d %H:%M:%S'),
                                                       predict_team_name,
                                                       predict_team_id,
                                                       item['predict_team_stat'],
                                                       [item['player_stats'] for item in team_matches_json[index]['predict_players']],
                                                       item['opposing_team_name'],
                                                       item['opposing_team_id'],
                                                       item['opposing_team_stat'],
                                                       [item['player_stats'] for item in team_matches_json[index]['opposing_players']],
                                                       item['league_name'],
                                                       item['leagueid'],
                                                       item['cluster'], 
                                                       item['radiant_win'],
                                                       item['radiant'],
                                                       item['duration'], 
                                                       item['match_id'],
                                                       match_outcome(item['radiant_win'], item['radiant'])) for index, item in enumerate(team_matches_json)], 
                      columns = ['start_date','predict_team_name', 'predict_team_id', 'predict_team_stat', 'predict_players', 'opposing_team_name', 'opposing_team_id', 'opposing_team_stat', 'opposing_players', 'league_name', 'leagueid', 'cluster', 'radiant_win', 'radiant', 'duration', 'match_id', 'match_outcome'])
    
    return all_matches_df


In [70]:
og_team_matches_df = team_matches_df(team_og_matches_json, 'OG', 2586976)

In [71]:
og_team_matches_df.head()

,start_date,predict_team_name,predict_team_id,predict_team_stat,predict_players,opposing_team_name,opposing_team_id,opposing_team_stat,opposing_players,league_name,leagueid,cluster,radiant_win,radiant,duration,match_id,match_outcome
0,2022-08-28 20:37:20,OG,2586976,"[{'predict_team_wins_total': 817, 'predict_tea...","[{'rank_tier': 80, 'profile': {'account_id': 1...",Team Aster,6209166,"[{'opposing_team_wins_total': 496, 'opposing_t...","[{'rank_tier': 80, 'leaderboard_rank': 10, 'co...",ESL One Malaysia 2022 powered by Intel,14388,155,True,True,2455,6728572058,True
1,2022-08-28 19:19:19,OG,2586976,"[{'predict_team_wins_total': 817, 'predict_tea...","[{'solo_competitive_rank': 3333, 'competitive_...",Team Aster,6209166,"[{'opposing_team_wins_total': 496, 'opposing_t...","[{'solo_competitive_rank': 8916, 'competitive_...",ESL One Malaysia 2022 powered by Intel,14388,155,False,False,2996,6728463079,True
2,2022-08-28 18:03:26,OG,2586976,"[{'predict_team_wins_total': 817, 'predict_tea...","[{'solo_competitive_rank': 3333, 'profile': {'...",Team Aster,6209166,"[{'opposing_team_wins_total': 496, 'opposing_t...","[{'solo_competitive_rank': 8916, 'mmr_estimate...",ESL One Malaysia 2022 powered by Intel,14388,151,False,False,2776,6728333090,True
3,2022-08-28 15:18:42,OG,2586976,"[{'predict_team_wins_total': 817, 'predict_tea...","[{'leaderboard_rank': 41, 'profile': {'account...",Team Secret,1838315,"[{'opposing_team_wins_total': 993, 'opposing_t...","[{'profile': {'account_id': 87278757, 'persona...",ESL One Malaysia 2022 powered by Intel,14388,151,False,False,1628,6728136960,True
4,2022-08-28 14:04:20,OG,2586976,"[{'predict_team_wins_total': 817, 'predict_tea...","[{'solo_competitive_rank': 3333, 'competitive_...",Team Secret,1838315,"[{'opposing_team_wins_total': 993, 'opposing_t...","[{'solo_competitive_rank': None, 'rank_tier': ...",ESL One Malaysia 2022 powered by Intel,14388,156,False,False,2547,6728055442,True


In [122]:
test = og_team_matches_df.iloc[0:6]
test

,start_date,predict_team_name,predict_team_id,predict_team_stat,predict_players,opposing_team_name,opposing_team_id,opposing_team_stat,opposing_players,league_name,leagueid,cluster,radiant_win,radiant,duration,match_id,match_outcome
0,2022-08-28 20:37:20,OG,2586976,"[{'predict_team_wins_total': 817, 'predict_tea...","[{'rank_tier': 80, 'profile': {'account_id': 1...",Team Aster,6209166,"[{'opposing_team_wins_total': 496, 'opposing_t...","[{'rank_tier': 80, 'leaderboard_rank': 10, 'co...",ESL One Malaysia 2022 powered by Intel,14388,155,True,True,2455,6728572058,True
1,2022-08-28 19:19:19,OG,2586976,"[{'predict_team_wins_total': 817, 'predict_tea...","[{'solo_competitive_rank': 3333, 'competitive_...",Team Aster,6209166,"[{'opposing_team_wins_total': 496, 'opposing_t...","[{'solo_competitive_rank': 8916, 'competitive_...",ESL One Malaysia 2022 powered by Intel,14388,155,False,False,2996,6728463079,True
2,2022-08-28 18:03:26,OG,2586976,"[{'predict_team_wins_total': 817, 'predict_tea...","[{'solo_competitive_rank': 3333, 'profile': {'...",Team Aster,6209166,"[{'opposing_team_wins_total': 496, 'opposing_t...","[{'solo_competitive_rank': 8916, 'mmr_estimate...",ESL One Malaysia 2022 powered by Intel,14388,151,False,False,2776,6728333090,True
3,2022-08-28 15:18:42,OG,2586976,"[{'predict_team_wins_total': 817, 'predict_tea...","[{'leaderboard_rank': 41, 'profile': {'account...",Team Secret,1838315,"[{'opposing_team_wins_total': 993, 'opposing_t...","[{'profile': {'account_id': 87278757, 'persona...",ESL One Malaysia 2022 powered by Intel,14388,151,False,False,1628,6728136960,True
4,2022-08-28 14:04:20,OG,2586976,"[{'predict_team_wins_total': 817, 'predict_tea...","[{'solo_competitive_rank': 3333, 'competitive_...",Team Secret,1838315,"[{'opposing_team_wins_total': 993, 'opposing_t...","[{'solo_competitive_rank': None, 'rank_tier': ...",ESL One Malaysia 2022 powered by Intel,14388,156,False,False,2547,6728055442,True
5,2022-08-27 23:42:16,OG,2586976,"[{'predict_team_wins_total': 817, 'predict_tea...","[{'solo_competitive_rank': 3333, 'competitive_...",Fnatic,350190,"[{'opposing_team_wins_total': 1153, 'opposing_...","[{'profile': {'account_id': 100471531, 'person...",ESL One Malaysia 2022 powered by Intel,14388,152,False,False,1931,6727040019,True


In [123]:
test.opposing_players[0]

[{'rank_tier': 80,
  'leaderboard_rank': 10,
  'competitive_rank': None,
  'solo_competitive_rank': 8916,
  'profile': {'account_id': 148215639,
   'personaname': 'Money',
   'name': 'Monet',
   'plus': True,
   'cheese': 0,
   'steamid': '76561198108481367',
   'avatar': 'https://avatars.akamai.steamstatic.com/343dab39597de5d25d02eab2b2fe48d8dde6ae0e.jpg',
   'avatarmedium': 'https://avatars.akamai.steamstatic.com/343dab39597de5d25d02eab2b2fe48d8dde6ae0e_medium.jpg',
   'avatarfull': 'https://avatars.akamai.steamstatic.com/343dab39597de5d25d02eab2b2fe48d8dde6ae0e_full.jpg',
   'profileurl': 'https://steamcommunity.com/id/Monetttt/',
   'last_login': '2016-02-13T17:13:10.687Z',
   'loccountrycode': None,
   'status': None,
   'is_contributor': False,
   'is_subscriber': False},
  'mmr_estimate': {'estimate': 7297}},
 {'profile': {'account_id': 129958758,
   'personaname': 'XTF',
   'name': 'Xxs',
   'plus': True,
   'cheese': 0,
   'steamid': '76561198090224486',
   'avatar': 'https://

In [131]:
test.apply(lambda x: x.predict_players[0]['profile']['account_id'], axis = 1)

0    183719386
1    183719386
2    183719386
3    183719386
4    183719386
5    183719386
dtype: int64

In [129]:
for index in range(5):

    test['opposing_players_id'] = test.apply(lambda x: x.predict_players[index]['profile']['account_id'], axis = 1)  

C:\Users\johnp\AppData\Local\Temp\ipykernel_24516\892139753.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test['opposing_players_id'] = test.apply(lambda x: x.predict_players[index]['profile']['account_id'], axis = 1)
C:\Users\johnp\AppData\Local\Temp\ipykernel_24516\892139753.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test['opposing_players_id'] = test.apply(lambda x: x.predict_players[index]['profile']['account_id'], axis = 1)
C:\Users\johnp\AppData\Local\Temp\ipykernel_24516\892139753.py:3

In [130]:
test

,start_date,predict_team_name,predict_team_id,predict_team_stat,predict_players,opposing_team_name,opposing_team_id,opposing_team_stat,opposing_players,league_name,leagueid,cluster,radiant_win,radiant,duration,match_id,match_outcome,opposing_players_id
0,2022-08-28 20:37:20,OG,2586976,"[{'predict_team_wins_total': 817, 'predict_tea...","[{'rank_tier': 80, 'profile': {'account_id': 1...",Team Aster,6209166,"[{'opposing_team_wins_total': 496, 'opposing_t...","[{'rank_tier': 80, 'leaderboard_rank': 10, 'co...",ESL One Malaysia 2022 powered by Intel,14388,155,True,True,2455,6728572058,True,401792574
1,2022-08-28 19:19:19,OG,2586976,"[{'predict_team_wins_total': 817, 'predict_tea...","[{'solo_competitive_rank': 3333, 'competitive_...",Team Aster,6209166,"[{'opposing_team_wins_total': 496, 'opposing_t...","[{'solo_competitive_rank': 8916, 'competitive_...",ESL One Malaysia 2022 powered by Intel,14388,155,False,False,2996,6728463079,True,401792574
2,2022-08-28 18:03:26,OG,2586976,"[{'predict_team_wins_total': 817, 'predict_tea...","[{'solo_competitive_rank': 3333, 'profile': {'...",Team Aster,6209166,"[{'opposing_team_wins_total': 496, 'opposing_t...","[{'solo_competitive_rank': 8916, 'mmr_estimate...",ESL One Malaysia 2022 powered by Intel,14388,151,False,False,2776,6728333090,True,401792574
3,2022-08-28 15:18:42,OG,2586976,"[{'predict_team_wins_total': 817, 'predict_tea...","[{'leaderboard_rank': 41, 'profile': {'account...",Team Secret,1838315,"[{'opposing_team_wins_total': 993, 'opposing_t...","[{'profile': {'account_id': 87278757, 'persona...",ESL One Malaysia 2022 powered by Intel,14388,151,False,False,1628,6728136960,True,401792574
4,2022-08-28 14:04:20,OG,2586976,"[{'predict_team_wins_total': 817, 'predict_tea...","[{'solo_competitive_rank': 3333, 'competitive_...",Team Secret,1838315,"[{'opposing_team_wins_total': 993, 'opposing_t...","[{'solo_competitive_rank': None, 'rank_tier': ...",ESL One Malaysia 2022 powered by Intel,14388,156,False,False,2547,6728055442,True,401792574
5,2022-08-27 23:42:16,OG,2586976,"[{'predict_team_wins_total': 817, 'predict_tea...","[{'solo_competitive_rank': 3333, 'competitive_...",Fnatic,350190,"[{'opposing_team_wins_total': 1153, 'opposing_...","[{'profile': {'account_id': 100471531, 'person...",ESL One Malaysia 2022 powered by Intel,14388,152,False,False,1931,6727040019,True,401792574


In [10]:
og_team_matches_df['opposing_team_wins_total'] = og_team_matches_df.apply(lambda x : x.opposing_team_stat[0]['opposing_team_wins_total'], axis = 1)
og_team_matches_df['opposing_team_losses_total'] = og_team_matches_df.apply(lambda x : x.opposing_team_stat[0]['opposing_team_losses_total'], axis = 1)
og_team_matches_df['opposing_team_rating'] = og_team_matches_df.apply(lambda x : x.opposing_team_stat[0]['opposing_team_rating'], axis = 1)

og_team_matches_df['predict_team_wins_total'] = og_team_matches_df.apply(lambda x : x.predict_team_stat[0]['predict_team_wins_total'], axis = 1)
og_team_matches_df['predict_team_losses_total'] = og_team_matches_df.apply(lambda x : x.predict_team_stat[0]['predict_team_losses_total'], axis = 1)
og_team_matches_df['predict_team_rating'] = og_team_matches_df.apply(lambda x : x.predict_team_stat[0]['predict_team_rating'], axis = 1)



In [49]:
og_team_matches_df.shape[0]

1335

In [11]:
og_team_matches_df.drop(['opposing_team_stat', 'predict_team_stat'], axis = 1, inplace = True)

In [12]:
og_team_matches_df.head()

,start_date,predict_team_name,predict_team_id,opposing_team_name,opposing_team_id,league_name,leagueid,cluster,radiant_win,radiant,duration,match_id,match_outcome,opposing_team_wins_total,opposing_team_losses_total,opposing_team_rating,predict_team_wins_total,predict_team_losses_total,predict_team_rating
0,2022-08-28 20:37:20,OG,2586976,Team Aster,6209166,ESL One Malaysia 2022 powered by Intel,14388,155,True,True,2455,6728572058,True,496,407,1493.25,817,508,1554.93
1,2022-08-28 19:19:19,OG,2586976,Team Aster,6209166,ESL One Malaysia 2022 powered by Intel,14388,155,False,False,2996,6728463079,True,496,407,1493.25,817,508,1554.93
2,2022-08-28 18:03:26,OG,2586976,Team Aster,6209166,ESL One Malaysia 2022 powered by Intel,14388,151,False,False,2776,6728333090,True,496,407,1493.25,817,508,1554.93
3,2022-08-28 15:18:42,OG,2586976,Team Secret,1838315,ESL One Malaysia 2022 powered by Intel,14388,151,False,False,1628,6728136960,True,993,496,1405.11,817,508,1554.93
4,2022-08-28 14:04:20,OG,2586976,Team Secret,1838315,ESL One Malaysia 2022 powered by Intel,14388,156,False,False,2547,6728055442,True,993,496,1405.11,817,508,1554.93


In [13]:
def combine_team_matches_data(team_matches_json, predict_team_id):  
    matches_total = len(team_matches_json)
    count = 1
    
    # 1 x API CALL
    predict_team_stat = get_team_stat(predict_team_id)
    
    for index, value in enumerate(team_matches_json):
        
         # Check if we already maxed out our requests for that minute (only 60 requests per minute on the free tier)
        if (count%5 == 0):
            for i in range(60, -1, -1):
                print('Exceeded max API calls, please wait:', i, 'seconds                                                ', end = '\r')
                time.sleep(1)
                
            count = 1

        else:

            match_info = team_matches_json[index]
            
            # 1 x API CALL 
            match_info_detailed = get_match_info(match_info['match_id'])

            # 5 x API CALL (5 players)
            predict_players = [{'account_id' : player['account_id'], 
                                'name' : player['name'],
                                'player_stats' : get_player_info(player['account_id'])
                               } for player in match_info_detailed['players'] if player['isRadiant'] == match_info['radiant']]

            # 5 x API CALL (5 players)
            opposing_players = [{'account_id' : player['account_id'], 
                                 'name' : player['name'],
                                 'player_stats' : get_player_info(player['account_id'])
                                } for player in match_info_detailed['players'] if player['isRadiant'] != match_info['radiant']]

            team_matches_json[index]['predict_players'] = predict_players
            team_matches_json[index]['opposing_players'] = opposing_players

            # 1 x API CALL
            opposing_team_stat = get_team_stat(match_info['opposing_team_id'])
            
            opposing_team = [{'opposing_team_wins_total' : opposing_team_stat['wins'], 
                              'opposing_team_losses_total' : opposing_team_stat['losses'], 
                              'opposing_team_rating' : opposing_team_stat['rating']}]
            
            predict_team = [{'predict_team_wins_total' : predict_team_stat['wins'], 
                              'predict_team_losses_total' : predict_team_stat['losses'], 
                              'predict_team_rating' : predict_team_stat['rating'],
                              'predict_team_name' : predict_team_stat['name'], 
                              'predict_team_id' : predict_team_stat['team_id']}]
            
            team_matches_json[index]['opposing_team_stat'] = opposing_team
            team_matches_json[index]['predict_team_stat'] = predict_team

            print('match number:', index, 'downloaded', matches_total - index, 'matches left to go', round((index / matches_total) * 100, 2),'% complete', end = '\r')
            
            count += 1
         
    return team_matches_json

In [14]:
def get_match_info(match_id):
    response = requests.get(BASE_URL + MATCHES + '/' + str(match_id))

    try:
        match_info_json = json.loads(response.content.decode('utf-8'))

    except requests.exceptions.Timeout:
        print('request is taking too long to complete, possible timeout, getting old data from Firebase')

    except requests.exceptions.RequestException as e:
        raise SystemExit(e)
        
    else:
        return match_info_json
    
    return 0

In [15]:
def get_player_info(account_id):
    response = requests.get(BASE_URL + PLAYERS + '/' + str(account_id))

    try:
        player_data_json = json.loads(response.content.decode('utf-8'))

    except requests.exceptions.Timeout:
        print('request is taking too long to complete, possible timeout, getting old data from Firebase')

    except requests.exceptions.RequestException as e:
        raise SystemExit(e)
    
    else:
        return player_data_json
    
    return 0

In [16]:
def get_team_stat(team_id):
    response = requests.get(BASE_URL + TEAMS + '/' + str(team_id))

    try:
        team_stat_json = json.loads(response.content.decode('utf-8'))

    except requests.exceptions.Timeout:
        print('request is taking too long to complete, possible timeout, getting old data from Firebase')

    except requests.exceptions.RequestException as e:
        raise SystemExit(e)
    
    else:
        return team_stat_json
    
    return 0

with open('team_og_matches.json', 'w', encoding='utf-8') as f:
    json.dump(team_og_matches_json, f, ensure_ascii=False, indent=4)